**Сергей, в связи с производительностью моего компа не успеваю уложиться в срок, хочется вникнуть в работу сети и проверить её работу при разных конфигурациях. Пока ставлю заглушку и по готовности работы вам сообщу.**